In [2]:
import numpy as np
import pandas as pd
from pyquery import PyQuery as pq
import time
import datetime
import requests
import urllib.request
import os
import calendar
import pickle
from lxml import etree

In [5]:
with open('search_addr_final.pkl','rb') as f:
    search_addr = pickle.load(f)

In [6]:
tickers = pd.read_csv('all_ticker.csv').iloc[:,1].values

In [8]:
with open('./10-Q_url/company_ft_dic_final.pkl','rb') as f:
    company_ft_dic = pickle.load(f)

In [3]:
def crawl_fp_txt_url(company_url):
    finance_report_url = []
    res = requests.get(company_url)
    doc = pq(res.text)

    temp = 'https://www.sec.gov'
    for i,d in enumerate(doc('.tableFile2 #documentsbutton').items()):

        f_url = temp+d.attr('href')    
        lv1_res = requests.get(f_url)
        lv1_doc = pq(lv1_res.text)
        txt_num = lv1_doc('#secNum').text().split(' ')[-1]
        sel = etree.HTML(lv1_res.text)
        txt_url = sel.xpath(f"//a[contains(text(),'{txt_num}.txt')] ")[0].attrib['href']
        finance_report_url.append(txt_url)
        
    if (len(finance_report_url) == 0):
        finance_report_url = 'Unfounded'
        
    return finance_report_url

In [4]:
def get_finace_report_txt(finance_report_url):
    # check directory
    if not os.path.isdir('./data'):
        os.mkdir('./data')
    
    for url in finance_report_url:
        symbol = url.split('/')[6] # number of company
        year = url.split('-')[1]  # 94 ~ 18
        filename = './data/' + symbol + '_' + year + 'txt'
        urllib.request.urlretrieve(url,filename)
        print(filename)

In [10]:
def get_all_fp_url(tickers,company_addr):
    # check directory
    if not os.path.isdir('./fp_url'):
        os.mkdir('./fp_url')
    
    # check files
    files = os.listdir('./fp_url')
    max_ver = 0

    for f in files:
        if 'company' in f:
            version = int(f.split('_')[3].split('.')[0])
            if (version > max_ver ):
                max_ver = version
                max_f = f
    if (max_ver == 0):     
        company_fp_dic = {}
    else:
        if (os.path.getsize(f"./fp_url/{max_f}") > 0):
            with open(f'./fp_url/{max_f}','rb') as f:
                company_fp_dic = pickle.load(f)
        else:
            company_fp_dic = {}
    length = len(company_fp_dic)   
    
    for i,key in enumerate(tickers[length:]):
        addr = company_addr[key]
        
        if (addr != 'Unfounded'):
            fp_addr = crawl_fp_txt_url(addr)
            company_fp_dic[key] = fp_addr
        else:
            company_fp_dic[key] = 'Unfounded'
            
        if(len(company_fp_dic)%1000 == 0):
            num = len(company_fp_dic)/1000

            with open(f"./fp_url/company_fp_dic_{num}.pkl","wb") as f:
                pickle.dump(company_fp_dic,f)
                print(f'./fp_url/company_fp_dic_{num}.pkl',i+length)
                
    with open(f"./fp_url/company_fp_dic_final.pkl","wb") as f:
        pickle.dump(company_fp_dic,f)
        print(f'./fp_url/company_fp_dic_final.pkl',i+length)

In [ ]:
get_all_fp_url(tickers,company_ft_dic)

./fp_url/company_fp_dic_1.0.pkl 999
